# Тестовое задание

`Мякишев Никита`

In [1]:
import pandas as pd
import numpy as np
import sqlite3 as sql

In [2]:
con = sql.connect('DB.db')

## Задание 1

- Напишите запрос, возвращающий строки с актуальными ценами (с наибольшей датой - date_start) для каждого товара (product). Посчитайте среднюю цену и запишите ответ-число в файл res1.txt

In [3]:
df = pd.read_sql_query('SELECT * \
                       FROM prices \
                       GROUP BY product \
                       HAVING date_start = MAX(date_start)', con)

In [4]:
price_mean = df.price.mean()

In [5]:
ouf = open('res1.txt', 'w')
ouf.write(f'{price_mean}')
ouf.close()

- Напишите функцию на любом языке программирования (предпочтительно Python) принимающую название продукта (поле product) и возвращающую tuple вида ([date_start1, date_start2,... ],[price1, price2,...]), из таблицы “prices”. Где date_start(n) - это даты, отсортированные по возрастанию, а price(n) - соответствующая дате цена на продукт. При каждом вызове функция должна обращаться к БД, выбирая необходимые данные. Формат дат особого значения не имеет.


In [6]:
def foo(product):
    connect = sql.connect('DB.db')
    df = pd.read_sql_query(f'SELECT date_start, price \
                            FROM prices \
                            WHERE product = "{product}" \
                            ORDER BY date_start', connect)
    connect.close()
    return (df.date_start.to_list(), df.price.to_list())

In [7]:
foo('A')

(['1970-01-01',
  '2010-01-21',
  '2010-08-17',
  '2010-11-02',
  '2012-08-27',
  '2012-12-06',
  '2013-01-11',
  '2013-05-02',
  '2013-05-08',
  '2013-11-01',
  '2014-08-30',
  '2015-12-29',
  '2016-05-04',
  '2016-08-14',
  '2016-09-28',
  '2017-04-01'],
 [297.79,
  164.68,
  558.58,
  127.08,
  352.13,
  0.35,
  100.61,
  120.5,
  260.19,
  1747.84,
  59.84,
  118.27,
  550.62,
  339.35,
  210.19,
  111.23])

## Задание 2

- Напишите запрос, результатом которого станет создание таблицы “revenue” вида:
product
date
amount
revenue

Где revenue вычисляется по формуле: revenue = sales.amount * prices.price. Естественно, price из таблицы prices должен соответствовать date из таблицы sales. Следует учитывать, что таблица sales содержит большое кол-во строк.
Выполните запрос и создайте таблицу в revenue "DB.db".

In [8]:
con = sql.connect('DB.db')
sales = pd.read_sql_query('SELECT * \
                           FROM sales', con)
prices = pd.read_sql_query('SELECT * \
                           FROM prices', con)

In [9]:
#pandas way
#revenue = sales
#revenue['revenue'] = revenue.apply(axis=1, func=lambda x: prices[(prices['product'] == x['product']) & (prices['date_start'] <= x['date'])].
#            sort_values('date_start', ascending=False).iloc[0,:]['price'] * x['amount'])

In [10]:
#sql way( faster, then pandas way c: )
revenue = pd.read_sql_query('SELECT sl.product, sl.date, sl.amount, sl.amount * pr.price as revenue \
                                 FROM sales as sl join prices as pr \
                                 ON sl.product = pr.product \
                                 WHERE pr.date_start = (SELECT MAX(date_start) \
                                                        FROM prices as pr1 \
                                                        WHERE pr1.product = sl.product and pr1.date_start <= sl.date)', con)

In [11]:
#adding table to database
revenue.to_sql('revenue', con, if_exists='replace')

- Запишите величину выборочного стандартного отклонения revenue для продукта "B" в файл res2.txt.

In [12]:
samp = revenue[revenue['product'] == 'B']
n = samp.shape[0]
x_dash = samp['revenue'].sum() / n
disp = samp['revenue'].apply(lambda x: (x - x_dash) ** 2).sum() / n
sd = (n * disp / (n - 1)) ** 0.5
print(f'x_dash = {x_dash}\ndispersion = {disp}\nsd = {sd}')


x_dash = 19030.889614074917
dispersion = 1119651220.3286636
sd = 33465.93838814805


In [13]:
ouf = open('res2.txt', 'w')
ouf.write(f'{sd}')
ouf.close()

## Задание 3

- Напишите функцию, которая принимает название товара product и две даты: date_start,
date_end, тренирует (находит c МНК коэффициенты k, b) линейную модель вида y = kx+b и возвращает коэффициенты, k и b. 

В модели x - это номер дня от начала заданного периода (0,1,2,3...), а y - кол-во проданных единиц продукции (amount из таблицы sales или revenue). При каждом вызове функция должна обращаться к таблице “revenue” или "sales" из задания 2, выбирая необходимые данные. 

In [14]:
from datetime import date, datetime
from sklearn.linear_model import LinearRegression

In [15]:
def lin_reg(product, date_start, date_end):
    def bar(date_now):
        return (date(*map(int, date_now.split("-"))) - date(*map(int, date_start.split("-")))).days
    connect = sql.connect('DB.db')
    df = pd.read_sql_query(f'SELECT date, amount \
                            FROM sales \
                            WHERE product = "{product}" \
                            AND date >= "{date_start}" \
                            AND date <= "{date_end}"', connect)
    df['days'] = df['date'].apply(func=lambda x: bar(x))
    lr = LinearRegression().fit(df['days'].to_numpy().reshape((-1, 1)), df['amount'].to_numpy())
    return (lr.coef_[0], lr.intercept_)
    connect.close()

- Запишите коэффициенты, возвращаемые функцией при начальных аргументах: lin_reg(product='Q', date_start='2019-01-01', date_end='2019-03-31') в res3.txt через пробел.

In [16]:
lr = lin_reg(product='Q', date_start='2019-01-01', date_end='2019-03-31')

In [17]:
ouf = open('res3.txt', 'w')
ouf.write(f'{lr[0]} {lr[1]}')
ouf.close()